In [21]:
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics import adjusted_rand_score
from datetime import datetime

In [22]:
#create class Graph in order to compute similarity between communities
class Graph(object):
    def __init__(self, numNodes):
        # Use a dictionary to store edges to create a CSR matrix when needed
        self.numNodes = numNodes
        self.edges = []

    def addEdge(self, start, end):
        # Add an edge to the list of edges
        if (start, end) not in self.edges:
            self.edges.append((start, end))

    def removeEdge(self, start, end):
        # Remove an edge from the list of edges
        if (start, end) in self.edges:
            self.edges.remove((start, end))
        else: #check if edge exists to not compute error
            print(f"There is no edge between {start} and {end}")

    def containsEdge(self, start, end): #check if edge exists in general
        return (start, end) in self.edges

    def to_csr_matrix(self):
        # Create the CSR representation from the list of edges
        rows, cols = zip(*self.edges) if self.edges else ([], [])
        data = [1] * len(self.edges)
        csr_mat = csr_matrix((data, (rows, cols)), shape=(self.numNodes, self.numNodes))
        return csr_mat

    def __len__(self):
        return self.numNodes


In [23]:
# # Function to create adjacency matrix for a community in CSR format to save space!
# def create_adjacency_matrix(community, max_size):
#     # Find all unique nodes in the community
#     unique_nodes = list(set(node for edge in community for node in edge)) 
#     node_to_index = {node: i for i, node in enumerate(unique_nodes)} #need to have a list of indices

#     # Initialize lists for the CSR representation
#     rows = []
#     cols = []
#     data = []

#     # Populate rows, cols, and data lists for non-zero elements
#     for node1, node2 in community:
#         index1 = node_to_index[node1]
#         index2 = node_to_index[node2]
#         rows.append(index1)
#         cols.append(index2)
#         data.append(1)

#     # Create a CSR matrix using scipy
#     actual_size = len(unique_nodes)
#     csr_mat = csr_matrix((data, (rows, cols)), shape=(actual_size, actual_size))

#     # If the actual size is smaller than max_size, pad the matrix with zeros
#     if actual_size < max_size + 1:
#         csr_mat.resize((max_size + 1, max_size + 1))

    # Function to create adjacency matrix for a community in CSR format with weighted values of duration
def create_adjacency_matrix(community, max_size):
    # Find all unique nodes in the community
    unique_nodes = list(set(node for edge in community for node in edge[:2]))
    node_to_index = {node: i for i, node in enumerate(unique_nodes)} #need to have a list of indices

    # Initialize lists for the CSR representation
    rows = []
    cols = []
    data = []

    # Populate rows, cols, and data lists for non-zero elements
    for node1, node2, duration in community:
        if node1 in node_to_index and node2 in node_to_index:
            index1 = node_to_index[node1]
            index2 = node_to_index[node2]

            # Use the provided duration as the weight (convert to integer if necessary)
            rows.append(index1)
            cols.append(index2)
            data.append(int(duration))

    # Create a CSR matrix using scipy
    actual_size = max_size + 1
    csr_mat = csr_matrix((data, (rows, cols)), shape=(actual_size, actual_size))
    return csr_mat


In [91]:
from sklearn.metrics.pairwise import cosine_similarity
#ARI is particularly useful here because it is adjusted for chance. 
# random similarities between matrices are not overemphasized, giving a more reliable measure of similarity.

# def compare_matrices(matrix1, matrix2, comparison_method="adjusted.rand"):
#     #converts the CSR matrices (sparse representations of weighted adjacency matrices) 
#     # to dense arrays, which are then flattened into 1D vectors.
#     dense_matrix1 = matrix1.toarray().flatten()
#     dense_matrix2 = matrix2.toarray().flatten()
#     # this vector represents whether nodes are connected and the weight of the connection, 
#     # using the provided duration values as edge weights.
#     # Use the selected comparison method
#     if comparison_method == "adjusted.rand":
#         similarity = adjusted_rand_score(dense_matrix1, dense_matrix2) #ARI compares these vectors, treating them as cluster labels.
#         #ARI evaluates how well the two communities match in terms of both structure (which nodes are connected) 
#         #and the weights of these connections.
#     else:
#         raise ValueError(f"Unsupported comparison method: {comparison_method}")

#     return similarity
'''
def compare_matrices(matrix1, matrix2, comparison_method="cosine"):
    # Converts the CSR matrices (sparse representations of weighted adjacency matrices)
    # to dense arrays, which are then flattened into 1D vectors.
    dense_matrix1 = matrix1.toarray().flatten()
    dense_matrix2 = matrix2.toarray().flatten()

    # Use cosine similarity as the comparison method
    dense_matrix1 = dense_matrix1.reshape(1, -1)  # Reshape for cosine similarity calculation
    dense_matrix2 = dense_matrix2.reshape(1, -1)
    similarity = cosine_similarity(dense_matrix1, dense_matrix2)[0][0]

    return similarity
'''
from scipy.sparse import csr_matrix

def compute_similarity_matrix_csr(A, B, max_iter=100, tol=1e-6):
    """
    Compute the similarity matrix between two CSR adjacency matrices A and B using iterative updates.
    """
    # Initialize the similarity matrix with ones (dense format)
    nA, nB = A.shape[0], B.shape[0]
    X = np.ones((nA, nB))
    
    # Iterative updates
    for _ in range(max_iter):
        X_new = B @ X @ A.T + B.T @ X @ A
        # Normalize using Frobenius norm
        X_new /= np.linalg.norm(X_new, ord='fro')
        
        # Check for convergence
        if np.linalg.norm(X_new - X, ord='fro') < tol:
            break
        
        X = X_new
    
    return X

def compute_community_similarity_csr(comm1, comm2):
    """
    Compute similarity score between two communities, where comm1 and comm2 are CSR adjacency matrices.
    """
    # Convert the CSR matrices to dense format temporarily for processing
    comm1_dense = comm1.toarray()
    comm2_dense = comm2.toarray()

    # Compute the similarity matrix
    similarity_matrix = compute_similarity_matrix_csr(comm1_dense, comm2_dense)
    
    # Return the Frobenius norm of the similarity matrix as the score
    similarity_score = np.linalg.norm(similarity_matrix, ord='fro')
    
    return similarity_score

# Example usage with CSR matrices for communities A and B



# Test the function to ensure the cosine similarity is now correctly implemented
#ARI can indirectly assess the degree of similarity between the edge weights. 
# If two matrices have similar edge structures but different weights, this will reflect in a lower ARI score.

#ARI helps determine both structural similarity (which nodes are connected) 
# and weighted edge similarity (how strong the connections are).

***Structural and Weighted Edge Similarity in This Code***

The code provides both structural similarity and weighted edge similarity through the following steps:

Structural Similarity:

The adjacency matrix encodes which nodes are connected. 
By converting these matrices into vectors, **the positions of non-zero elements in these vectors represent the connections between nodes**.
This code evaluates whether the matrices agree on the existence of edges between nodes, giving structural similarity.

Weighted Edge Similarity:

The weight of the edges is represented by the duration values in each tuple. 
When constructing the CSR matrix, **these durations become the actual values in the matrix**, rather than just indicating the presence or absence of a connection. (The code of this is still available, if we want to do 50/50 similarity comparison)

By flattening the weighted adjacency matrix, ARI takes into account the weights when comparing two vectors. 
If the durations are different, it affects the overall similarity score. 
This reflects how strong or weak the edges are between corresponding nodes.
Thus, ARI provides a metric for how similar the two graphs are not only based on their topology but also on the intensity of the connections.

***Advantages***

Handles Weighted Comparisons:

In this core it **compares weighted vectors derived from adjacency matrices**.


Robust Against Random Similarities:

Since the ARI is adjusted for chance, it is more **robust against random structural similarities between different graphs**. This property makes it well-suited for assessing similarity between communities where the goal is to identify truly similar interaction patterns rather than coincidental connections.

Single Measure of Similarity:

ARI gives a single score that encapsulates both structural and weight similarities, making it a convenient metric for evaluating the overall similarity of two graphs.


In [92]:
def max_list(list):
    list_len = [len(i) for i in list]
    return max(list_len)

def print_similarity_score(communities):
    # Find the maximum node index across all communities
    max_node = max(max(edge[0], edge[1]) for community in communities for edge in community)
    
    # Create adjacency matrices for each community
    matrices = [create_adjacency_matrix(community, max_node) for community in communities]
    
    # Loop through all pairs of communities and print similarity scores
    for i, matrix1 in enumerate(matrices):
        for j in range(i + 1, len(matrices)):  # Ensure j is within the valid range
            matrix2 = matrices[j]
            similarity = compute_community_similarity_csr(matrix1, matrix2)
            print(f"Score similarity between community {i} and {j} is:")
            print(similarity)

def group_similar_communities(communities, threshold, max_size):
    # Find the maximum node index across all communities
    max_node = max(max(edge[0], edge[1]) for community in communities for edge in community)
    
    # Create CSR adjacency matrices for each community
    matrices = [create_adjacency_matrix(community, max_node) for community in communities]
    
    # Initialize a list to hold grouped communities
    groups = []
    
    # List to track communities that have been grouped
    grouped = set()
    
    for i, matrix1 in enumerate(matrices):
        if i in grouped:
            continue
        current_group = [communities[i]]
        
        for j in range(i + 1, min(len(matrices), max_size)):  # Compare i with every community after i, up to max_size
            if j not in grouped:
                matrix2 = matrices[j]
                # Compute similarity between the two communities' adjacency matrices
                similarity = compute_community_similarity_csr(matrix1, matrix2)  # CSR-based similarity function
                
                # Check if the similarity exceeds the threshold
                if similarity >= threshold:
                    current_group.append(communities[j])
                    grouped.add(j)  # Mark as grouped to avoid reprocessing
        
        groups.append(current_group)

    return groups
# def group_similar_communities(communities, threshold):
#     # Find the maximum node index across all communities
#     max_node = max(max(edge[0], edge[1]) for community in communities for edge in community)
    
#     # Create adjacency matrices for each community
#     matrices = [create_adjacency_matrix(community, max_node) for community in communities]
    
#     # Initialize a list to hold grouped communities
#     groups = []
    
#     # List to track communities that have been grouped
#     grouped = set()
    
#     for i, matrix1 in enumerate(matrices):
#         if i in grouped:
#             continue
#         current_group = [communities[i]]
#         for j in range(i + 1, len(matrices)):  # Compare i with every community after i
#             matrix2 = matrices[j]
#             similarity = compare_matrices(matrix1, matrix2)
#             print(f"Score similarity between community {i} and {j} is:")
#             print(similarity)
#             if similarity >= threshold:
#                 current_group.append(communities[j])
#                 grouped.add(j)  # Mark as grouped to avoid reprocessing
#         groups.append(current_group)

#     # Ensure all remaining ungrouped communities are printed
#     for i in range(len(matrices)):
#         for j in range(i + 1, len(matrices)):
#             similarity = compare_matrices(matrices[i], matrices[j])
#             print(f"Score similarity between community {i} and {j} is:")
#             print(similarity)
    
#     return groups

# Run the function and test the similarity scoring





In [93]:
# Example of how you would use this with community data
# communities = [
#     [(0, 1), (1, 2)],        # Community 1
#     [(10, 11), (11, 13), (13, 14)],  # Community 2
#     [(4, 5), (5, 6)],         # Community 3
#     [(7, 8)],                 # Community 4
# ]

# communities = [
#     [(0, 1, '001530'), (1, 2, '001645')],                   # Community 1
#     [(10, 11, '001130'), (11, 13, '001245'), (13, 14, '001315')],  # Community 2
#     [(4, 5, '000945'), (5, 6, '001030')],                   # Community 3
#     [(7, 8, '011200')],                                     # Community 4
# ]
communities = [
    [(1, 2, '000002'), (2, 3, '000005')],  # Community 13
    [(4, 5, '000006'), (5, 6, '000015'), (6, 7, '000008')],  # Community 4
    [(8, 9, '000103'), (9, 10, '001005'), (10, 11, '000204')],  # Community 10
    [(12, 13, '000002'), (13,14,'000005'), (14, 12, '000008')]  # Community 1

]

print_similarity_score(communities)
# Group communities based on 60% similarity
similar_communities = group_similar_communities(communities, threshold=0.9, max_size=len(communities))

# Print the results
for idx,group in enumerate(similar_communities):
    print(f"Group {idx + 1}: {group}")


Score similarity between community 0 and 1 is:
0.9999999999999999
Score similarity between community 0 and 2 is:
1.0
Score similarity between community 0 and 3 is:
1.0
Score similarity between community 1 and 2 is:
0.9999999999999999
Score similarity between community 1 and 3 is:
0.9999999999999999
Score similarity between community 2 and 3 is:
1.0
Group 1: [[(1, 2, '000002'), (2, 3, '000005')], [(4, 5, '000006'), (5, 6, '000015'), (6, 7, '000008')], [(8, 9, '000103'), (9, 10, '001005'), (10, 11, '000204')], [(12, 13, '000002'), (13, 14, '000005'), (14, 12, '000008')]]
